<a href="https://colab.research.google.com/github/progsthegr8/projetHeart/blob/main/BPM_case_p4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import scipy.signal as spsig
import scipy.fftpack as spfft
import scipy.io.wavfile as spiow
from scipy import fftpack as spfft
import librosa ,librosa.display
import math
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from IPython.display import Audio, IFrame, display

Functions


In [ ]:
def bpm_to_hz(s):
  return s*0.01666667

def error(exp,thr):
  return ((abs(exp-thr))/exp)*100

def remove_brackets(s):
  flat_list=[]
  for element in s:
    if type(element) is list:
      for item in element:
        flat_list.append(item)
    else:
      flat_list.append(element)
  return flat_list



File upload

In [ ]:
DATABASE_PATH="/content/drive/MyDrive/database"

Control=False #True if the patient is a 'Co', False if it is 'Ca'
cas=["100","11","110","12","150","151","152","153","154","156","157","158","2","23","24","26","29","85"] # The id of the patient, for exemple '1'
point="P4"
my_tempo=[]
file_list=[]
hop_length = 512
if(Control):
    FileName=DATABASE_PATH+"/Contrôles/Centre 1/Pn_POA_Co"+cas+"/Pn_POA_Co"+cas+"_"+point+".wav"
    textPath=DATABASE_PATH+"/Contrôles/Centre 1/Pn_POA_Co"+cas+"/Pn_POA_Co"+cas+"_"+point+".txt"
else:
  for i in range (0,len(cas)):
    FileName=glob.glob(DATABASE_PATH+"/Cas/Centre 1/Pn_POA_Ca"+cas[i]+"/Pn_POA_Ca"+cas[i]+"_"+point+".wav")
    #textPath=DATABASE_PATH+"/Cas/Centre 1/Pn_POA_Ca"+cas+"/Pn_POA_Ca"+cas+"_"+point+".txt"
    file_list.append(FileName)
    print(FileName)
    for fl in range(len(FileName)):
      if(len(FileName)!=0):
        s,sr  = librosa.load(FileName[fl])
        onset_env = librosa.onset.onset_strength(s, sr=sr, hop_length=hop_length, n_fft=2048)
        frames=range(len(onset_env))
        t=librosa.frames_to_time(frames,sr=sr,hop_length=hop_length)
        fig, ax = plt.subplots()
        fig, ax1 = plt.subplots()
        fig, ax2 = plt.subplots()
        fig, ax3 = plt.subplots()

        ax.plot(t,onset_env)
        ax.set_xlim([0,t.max()])
        ax.set_ylim(0)
        ax.set(xlabel='Time (sec)')
        ax.set_title(label='Novelty Function')


        S=librosa.stft(onset_env,hop_length=1,n_fft=512)
        fr_tempogram=np.absolute(S)

        n0 = 100
        n1 = 500
        ax1.plot(t[n0:n1], onset_env[n0:n1])
        ax1.set_xlim(t[n0], t[n1])
        ax1.set(xlabel='Time (sec)')

        #plotting autocorr of the segment

        tmp = np.log1p(onset_env[n0:n1])
        r = librosa.autocorrelate(tmp)

        ax2.plot(t[:n1-n0], r)
        ax2.set_xlim(t[0], t[n1-n0])
        ax2.set_xlabel('Lag (sec)')
        ax2.set_ylim(0)

        ax3.plot(60/t[:n1-n0], r)
        ax3.set_xlim([20, 200])
        ax3.set_xlabel('Tempo (BPM)')
        ax3.set_ylim(0)

        plt.show()

        librosa.display.specshow(fr_tempogram, sr=sr, hop_length=hop_length, x_axis='time')

        tempogram = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr, hop_length=hop_length, win_length=400)

        tempo = librosa.beat.tempo(s, sr=sr)[0]
        my_tempo.append(tempo)
        a=plt.subplot()
        a.axhline(tempo, color='w', linestyle='--', alpha=1,label='Estimated tempo={:g}'.format(tempo))
        a.legend(loc='upper right')
        librosa.display.specshow(tempogram, sr=sr, hop_length=hop_length, x_axis='time', y_axis='tempo',ax=a)
        print(tempo,"bpm","=>",bpm_to_hz(tempo),"HZ")





In [ ]:
import pandas as pd
my_error=[]
for element in my_tempo:
  k=error(element,150)
  my_error.append(k)
new_file_list=remove_brackets(file_list)
zip_iterator=zip(new_file_list,my_tempo)
patient_df=dict(zip_iterator)
error_df=pd.DataFrame(my_error,columns=['error'])
df=pd.DataFrame(patient_df.items(), columns=['patient', 'tempo'])
frame=[df,error_df]
result=pd.concat(frame,axis=1)
print(result)


IGNORE 



In [ ]:
S=librosa.stft(onset_env,hop_length=1,n_fft=512)
fr_tempogram=np.absolute(S)

In [ ]:
librosa.display.specshow(fr_tempogram, sr=sr, hop_length=hop_length, x_axis='time')

Autocorrelation Tempogram

In [ ]:
n0 = 100
n1 = 500
plt.plot(t[n0:n1], onset_env[n0:n1])
plt.xlim(t[n0], t[n1])
plt.xlabel('Time (sec)')
plt.title('Novelty Function')

In [ ]:
#plotting autocorr of the segment
tmp = np.log1p(onset_env[n0:n1])
r = librosa.autocorrelate(tmp)

In [ ]:
plt.plot(t[:n1-n0], r)
plt.xlim(t[0], t[n1-n0])
plt.xlabel('Lag (sec)')
plt.ylim(0)

In [ ]:
plt.plot(60/t[:n1-n0], r)
plt.xlim(20, 200)
plt.xlabel('Tempo (BPM)')
plt.ylim(0)

In [ ]:
tempogram = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr, hop_length=hop_length, win_length=400)

In [ ]:
tempo = librosa.beat.tempo(s, sr=sr)[0]
a=plt.subplot()
a.axhline(tempo, color='w', linestyle='--', alpha=1,
            label='Estimated tempo={:g}'.format(tempo))
a.legend(loc='upper right')
librosa.display.specshow(tempogram, sr=sr, hop_length=hop_length, x_axis='time', y_axis='tempo',ax=a)


In [ ]:
print(tempo,"bpm","=>",bpm_to_hz(tempo),"HZ")

In [ ]:
def bpm_to_hz(s):
  return s*0.01666667